In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [3]:
# Read data (pdf) files
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,ServiceContext
documents=SimpleDirectoryReader("data/pdf").load_data()

In [4]:
#documents

In [5]:
# Convert this documents to index
service_context = ServiceContext.from_defaults(chunk_size=512)
index=VectorStoreIndex.from_documents(documents, service_context=service_context ,show_progress=True)
index

/var/folders/tt/s83h4n255y95sgyr2yrtv2x00000gp/T/ipykernel_73545/1848985020.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(chunk_size=512)


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
# Get the Query Engine from the Index
query_engine = index.as_query_engine()
query_engine

In [7]:
# Ask a query to this Query Engine
response = query_engine.query("Tell me something about Himanshu")
print(response)

Himanshu is from Odisha and spent his childhood in Puri. He completed his schooling at Puri Zila School and his intermediate education at SCS college. He pursued his engineering (B.Tech) from NMIET in Bhubaneswar. He began his career as a Systems Engineer Trainee at TCS before moving to Capco technologies Ltd for a brief period. Currently, he is working at Landmark Digital as a Technical Lead, where he has been for 6 years.


In [8]:
# See the responses (Top response along with top - x responses)
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response, show_source=True)

Final Response: Himanshu is from Odisha and spent his childhood in
Puri. He completed his schooling at Puri Zila School and his
intermediate education at SCS college. He pursued his engineering
(B.Tech) from NMIET in Bhubaneswar. He began his career as a Systems
Engineer Trainee at TCS before moving to Capco technologies Ltd for a
brief period. Currently, he is working at Landmark Digital as a
Technical Lead, where he has been for 6 years.
______________________________________________________________________
Source Node 1/1
Node ID: c6f2819a-e53f-45b3-9e5f-81aa901155cc
Similarity: 0.8536585389590541
Text: Himanshu belongs to Odisha. He has spent his childhood in Puri -
one of the beau9ful ci9es of Odisha. He has done his schooling from
Puri Zila School and has completed his intermediate from SCS college.
Post that he has done his engineering (B.Tech) from NMIET,
Bhubaneswar.   AHer this, he joined TCS as a Systems Engineer Trainee
where he starte...


## Create a query engine using Vector Retriever

In [9]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

retriever = VectorIndexRetriever(index=index, similarity_top_k=4)
post_processor = SimilarityPostprocessor(similarity_cutoff=0.80)
query_engine = RetrieverQueryEngine(retriever=retriever, node_postprocessors=[post_processor])


In [10]:
# Ask a query to this Query Engine
response = query_engine.query("Tell me something about Himanshu")
# See the responses (Top response along with top - 4 responses)
pprint_response(response, show_source=True)

Final Response: Himanshu is originally from Odisha and spent his
childhood in Puri. He completed his schooling from Puri Zila School
and his intermediate education from SCS college. He pursued his
engineering (B.Tech) from NMIET in Bhubaneswar. He began his career as
a Systems Engineer Trainee at TCS before moving to Capco technologies
Ltd for a brief period. Currently, he is working at Landmark Digital
as a Technical Lead, where he has been for 6 years.
______________________________________________________________________
Source Node 1/1
Node ID: c6f2819a-e53f-45b3-9e5f-81aa901155cc
Similarity: 0.8536585389590541
Text: Himanshu belongs to Odisha. He has spent his childhood in Puri -
one of the beau9ful ci9es of Odisha. He has done his schooling from
Puri Zila School and has completed his intermediate from SCS college.
Post that he has done his engineering (B.Tech) from NMIET,
Bhubaneswar.   AHer this, he joined TCS as a Systems Engineer Trainee
where he starte...


# Everything in one place

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)

INDEX_STORE = './index_storage'

if not os.path.exists(INDEX_STORE):
    # Load the documents, create the index and store it for later usage
    documents = SimpleDirectoryReader("data/pdf").load_data()
    index = VectorStoreIndex.from_documents(documents,show_progress=True)
    # Store the index
    index.storage_context.persist(persist_dir=INDEX_STORE)
else:
    # Load the index from file
    storage_context = StorageContext.from_defaults(persist_dir=INDEX_STORE)
    index = load_index_from_storage(storage_context=storage_context)


query_engine = index.as_query_engine()
response = query_engine.query("How many companies Himanshu has worked so far ?");
print(response)
response = query_engine.query("Does Himanshu belong to Africa ? Show the reason ");
print(response)

Himanshu has worked in three companies so far.
No, Himanshu does not belong to Africa. He belongs to Odisha, India, as mentioned in the context.
